<a href="https://colab.research.google.com/github/PlaZMaD/ml_miem_2024/blob/main/Seminar_7_Inference_BDA_resampling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Семинар 7. Статистический вывод. Байесовский анализ данных.  Методы семплирования


(использованы материалы курса [Машинное обучение в Питоне](https://www.hse.ru/edu/courses/450323352))

Дополнительная литература:
* [Учебник Computer Age Statistical Inference (CASI)](https://web.stanford.edu/~hastie/CASI_files/PDF/casi.pdf). Chapter 3. Bayesian Inference.

* [Учебник Bayesian Data Analysis (BDA)](https://users.aalto.fi/~ave/BDA3.pdf) и [курс BDA](https://github.com/avehtari/BDA_course_Aalto) от Aki Vehtari


# Статистический вывод (inference)

Основы фреквентистских, байесовских (и фишерианских) [выводов](https://ru.wikipedia.org/wiki/%D0%A1%D1%82%D0%B0%D1%82%D0%B8%D1%81%D1%82%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%B8%D0%B9_%D0%B2%D1%8B%D0%B2%D0%BE%D0%B4).

Статистический анализ:
* Алгоритм
  * Среднее значение: $\bar{x} = \sum\limits_{i=1}^n{\frac{x_i}{n}}$
* Вывод
  * Стандартная ошибка: $\widehat{\mathrm{se}} = \sqrt{\frac{\sum_{i=1}^n{(x_i-\bar{x})^2}}{n(n-1)}}$

"Вывод" касается не только точности: говоря в общем, алгоритмы говорят о том, что делает исследователь, в то время как вывод говорит о том, почему он или она это делает.

Конечно, $\widehat{\mathrm{se}}$ сам по себе является алгоритмом, который может быть (и является) предметом дальнейшего инференциального анализа относительно его точности.

## Пример 1. Линейная регрессия

Датасет 157 потенциальных доноров почки.
`age` - возраст
`tot` - некая метрика, соответствующая здоровью почки

In [ ]:
# Импортируем необходимые библиотеки
import matplotlib.pyplot as plt, seaborn as sns
from matplotlib.patches import Patch
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy.stats import beta

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import TimeSeriesSplit, KFold, ShuffleSplit, \
  StratifiedKFold, GroupShuffleSplit, GroupKFold, StratifiedShuffleSplit

from sklearn.pipeline import Pipeline

In [ ]:
# sns.set()
# plt.rcParams['figure.figsize'] = [16, 6]

In [ ]:
!wget --no-check-certificate https://web.stanford.edu/~hastie/CASI_files/DATA/kidney.txt

In [ ]:
# Загрузим датасет
df = pd.read_csv("kidney.txt", sep=' ')

In [ ]:
df.head()

In [ ]:
df.describe()

### Линейное фитирование

In [ ]:
ols_result = sm.OLS(df.tot, sm.add_constant(df.age)).fit()
print(ols_result.params)

In [ ]:
# https://stackoverflow.com/questions/54540030/to-calculate-the-se-fit-and-resdiual-scale-that-are-the-outputs-from-lm-fu

x_pred = np.linspace(20, 80, 7)
y_pred = ols_result.predict(sm.add_constant(x_pred))

covariance_matrix = ols_result.cov_params()
xO = pd.DataFrame({"Constant":np.ones(len(x_pred))}).join(pd.DataFrame(x_pred)).values
x1 = np.dot(xO, covariance_matrix)
y_pred_se = np.sqrt(np.sum(x1 * xO,axis = 1))

In [ ]:
fig, ax = plt.subplots(figsize=(12,8))

plt.scatter(df.age, df.tot, c='k', s=2)
plt.plot(x_pred, y_pred, c='g')
plt.errorbar(x_pred, y_pred, yerr=y_pred_se, linestyle='None', fmt='.r',  capsize=3,  ecolor='k')
plt.xlabel('age')
plt.ylabel('tot')
plt.grid()

### Полиномиальное фитирование

In [ ]:
lowess = sm.nonparametric.lowess
w = lowess(df.tot, df.age, frac=1./3)
# frac - доля данных, используемых  при оценке каждого значения y.

In [ ]:
fig, ax = plt.subplots(figsize=(12,8))

plt.scatter(df.age, df.tot, c='k', s=2)
plt.plot(w[:,0], w[:,1], c='g')
plt.xlabel('age')
plt.ylabel('tot')
plt.grid();

Не существует формулы, подобной стандартной ошибке, чтобы сделать вывод о точности такой кривой. Вместо этого можно использовать вычислительный механизм статистического вывода - бутстрап. Применение `lowess` к бутстрап-выборке создает бутстрап-репликацию исходного расчета.

In [ ]:
fig, ax = plt.subplots(figsize=(12,8))

plt.scatter(df.age, df.tot, c='k', s=2)

for i in range(10):
  df2 = df.sample(n=500, replace=True, random_state=i)
  w = lowess(df2.tot, df2.age, frac=1./3)
  plt.plot(w[:,0], w[:,1])

plt.xlabel('age')
plt.ylabel('tot')
plt.grid();

Изменчивость бутстрап-репликаций определяет точность исходной кривой.

# Байесовский анализ данных

## Условные обозначения
$\theta$ - ненаблюдаемые векторные величины или параметры системы

y - наблюдаемые данные

$\widetilde{y}$ - неизвестные, но потенциально наблюдаемые величины

*Плотность распределения* имеет тот же смысл, что и *распределение*.

Обычно мы используем греческие буквы для параметров, строчные латинские буквы для наблюдаемых или наблюдаемых скаляров и векторов (и иногда матриц), и строчные латинские буквы для наблюдаемых или наблюдаемых матриц.

$p(\theta|y)$ или $p(\widetilde{y}|y)$ - условные вероятности для наблюдаемого значения $y$.

$p(\cdot|\cdot)$ - условная плотность вероятности

$p(\cdot)$ - частотное (маргинальное) распределение

$\mathrm{Pr}(\cdot)$ - вероятность события

## Правило Байеса

Распределение **совместной вероятности** для $\theta$ и $y$:
$$p(\theta,y) = p(\theta) p(y|\theta)$$

где:

$p(\theta)$ - предварительное распределение,

$p(y|\theta)$ - распределение выборки (или распределение данных).

Правило Байеса определяет **апостериорное распределение**:
$$p(\theta|y) = \frac{p(\theta,y)}{p(y)} = \frac{p(\theta) p(y|\theta)}{p(y)}.$$

При выбранной вероятностной модели апостериорное распределение $p(\theta|y)$, рассматриваемое как функция $\theta$ для фиксированного $y$ есть функция правдоподобия.

## Некоторые широкораспространённые распределения (учебник BDA, Приложение A)

### Непрерывные:

| Распределение | Обозначения <img width=105/> | Параметры <img width=80/> |
| ----------- | --------- | ----------- |
| **Uniform** | $\theta \sim \mathrm{U}(\alpha, \beta)$<br />$p(\theta) = \mathrm{U}(\theta|\alpha,\beta)$ | boundaries $\alpha, \beta$ with $\beta > \alpha$ |
| **Normal**<br />**(Gaussian)** | $\theta \sim \mathrm{N}(\mu, \sigma^2)$<br />$p(\theta) = \mathrm{N}(\theta|\mu,\sigma^2)$ | location $\mu$ and scale $\sigma > 0$ |
| **Lognormal** | $\theta \sim \mathrm{lognorm}(\mu, \sigma^2)$<br />$p(\theta) = \mathrm{lognorm}(\theta|\mu,\sigma^2)$ | location $\mu$ and log-scale $\sigma > 0$ |
| **Multivariate normal** | $\theta \sim \mathrm{N}(\mu, \Sigma)$<br />$p(\theta) = \mathrm{N}(\theta|\mu,\Sigma)$<br />(implicit dimension $d$) | symmetric, pos. definite<br />$d\times d$ variance matrix $\Sigma$ |
| **Gamma** | $\theta \sim \mathrm{Gamma}(\alpha, \beta)$<br />$p(\theta) = \mathrm{Gamma}(\theta|\alpha, \beta)$ | shape $\alpha > 0$ and inverse scale $\beta > 0$ |
| **Chi-square** | $\theta \sim \chi_\nu^2$<br />$p(\theta) = \chi_\nu^2(\theta)$ | degrees of freedom $\nu > 0$ |
| **t**<br />**(Student-t)** | $\theta \sim \mathrm{t}_\nu(\mu,\sigma^2)$<br />$p(\theta) = \mathrm{t}_\nu(\theta|\mu,\sigma^2)$ | degrees of freedom $\nu > 0$<br />location $\mu$ and scale $\sigma > 0$ |
| **Beta** | $\theta \sim \mathrm{Beta}(\alpha, \beta)$<br />$p(\theta) = \mathrm{Beta}(\theta|\alpha, \beta)$ | 'prior sample sizes' $\alpha > 0$ and $\beta > 0$ |

### Дискретные:

| Распределение | Обозначения <img width=145/> | Параметры <img width=80/> |
| ----------- | --------- | ----------- |
| **Poisson** | $\theta \sim \mathrm{Poisson}(\lambda)$<br />$p(\theta) = \mathrm{Poisson}(\theta|\lambda)$ | 'rate' $\lambda > 0$ |
| **Binomial** | $\theta \sim \mathrm{Bin}(n, p)$<br />$p(\theta) = \mathrm{Bin}(\theta|n,p)$ | 'sample size' $n$ (positive integer)<br />'probability' $p\in[0,1]$ |
| **Bernoulli**<br />(Binomial with n=1) | $\theta \sim \mathrm{Bern}(p)$<br />$p(\theta) = \mathrm{Bern}(\theta|p)$ | 'probability' $p\in[0,1]$ |
| **Multinomial** | $\theta \sim \mathrm{Multin}(n; p_1,...,p_k)$<br />$p(\theta) = \mathrm{Multin}(\theta|n; p_1,...,p_k)$ | 'sample size' $n$ (positive integer)<br />'probabilities' $p_j\in[0,1]$<br />$\sum_{j=1}^k p_j = 1$ |

## Пример 1. Анализ с использованием равномерного априорного распределения

**Вероятность рождения девочки при предлежании плаценты у матери** (учебник BDA, стр. 37):

Исследование рожениц с предлежанием плаценты в Германии показало, что из `980` рождённых `437` были девочками. Насколько это подтверждает утверждение, что доля девочек, рождённых у матерей с предлежанием плаценты меньше, чем `0.485` = доли женщин в общей популяции?

Всего наблюдалось `437` девочек и `543` мальчика. Вычислите и постройте график апостериорного распределения доли девочек $\theta$, используя равномерное априорное распределение $\theta$.

Известно, что для равномерного априорного распределения [сопряжённым](https://ru.wikipedia.org/wiki/%D0%A1%D0%BE%D0%BF%D1%80%D1%8F%D0%B6%D1%91%D0%BD%D0%BD%D0%BE%D0%B5_%D0%B0%D0%BF%D1%80%D0%B8%D0%BE%D1%80%D0%BD%D0%BE%D0%B5_%D1%80%D0%B0%D1%81%D0%BF%D1%80%D0%B5%D0%B4%D0%B5%D0%BB%D0%B5%D0%BD%D0%B8%D0%B5) будет Бета-распределение. Поэтому апостериорная вероятность будет `Beta(437+1, 543+1)`.

In [ ]:
?beta

In [ ]:
dist = beta(438, 544)

Давайте пройдёмся по значениям вокруг искомой вероятности:

In [ ]:
x = np.linspace(0.36, 0.54, 80)

И найдём плотность вероятности для `Beta(438, 544)`:



In [ ]:
pdz = dist.pdf(x)

Построим соответствующее распределение:

In [ ]:
print('Равномерное априорное распределение -> Апостериорное Beta(438,544)-распределение')
fig, ax = plt.subplots(figsize=(12,6))
plt.plot(x, pdz)

plt.annotate(r'$p(\theta|y,n)$', (x[35] - 0.005, pdz[35]), ha='right')

plt.axvline(0.485, color='C1')
plt.annotate('Средняя доля женщин \nв популяции', (0.485 + 0.005, 14), ha='left')

# вычислим точки, попавшие в интервал между 2.5% и 97.5%
x_95_idx = (x > dist.ppf(0.025)) & (x < dist.ppf(0.975))
plt.fill_between(x[x_95_idx], pdz[x_95_idx], color='gray')
plt.text(dist.median(), 8, "95%", horizontalalignment='center')


plt.xlabel(r'$\theta$')
ax.axes.get_yaxis().set_visible(False)

## Пример 2. Иллюстрация влияния априорной информации в биномиальной модели

Проиллюстрируйте влияние априорной информации и сравните апостериорные распределения с различными значениями параметров для бета-распределения. Используйте данные из Примера 1.

In [ ]:
x = np.linspace(0.375, 0.525, 150)

# апостериорное распределение для данных (437,543) и равномерное априорное распределение Beta(1,1)
au = 438
bu = 544
# подсчитаем плотности вероятности (pdf)
pdu = beta.pdf(x, au, bu)

# сравним 3 случая различных априорных распределений
# Beta(0.485*n, (1-0.485)*n), для n = 2, 20, 200
ap = np.array([0.485 * (2*10**i) for i in range(3)])
bp = np.array([(1-0.485) * (2*10**i) for i in range(3)])
# corresponding posteriors with data (437,543)
ai = 437 + ap
bi = 543 + bp
# подсчитаем априорные и постериорные pdf-распределения
pdp = beta.pdf(x, ap[:,np.newaxis], bp[:,np.newaxis])
pdi = beta.pdf(x, ai[:,np.newaxis], bi[:,np.newaxis])

Построим эти распределения:

In [ ]:
# plot 3 subplots
fig, axes = plt.subplots(
    nrows=3, ncols=1, sharex=True, sharey=True, figsize=(8, 12))
# show only x-axis
# plot_tools.modify_axes.only_x(axes)
# manually adjust spacing
fig.subplots_adjust(hspace=0.4)

# 3 subplots
for i, ax in enumerate(axes):
    # plot three precalculated densities
    post1, = ax.plot(x, pdu, color='C1', linewidth=5)
    prior, = ax.plot(x, pdp[i], 'k:')
    post2, = ax.plot(x, pdi[i], color='k', dashes=(6, 8))
    # add vertical line
    known = ax.axvline(0.485, color='C0')
    # set the title for this subplot
    ax.set_title(
        r'$\alpha/(\alpha+\beta) = 0.485,\quad \alpha+\beta = {}$'
        .format(2*10**i)
    )
# limit x-axis
axes[0].autoscale(axis='x', tight=True)
axes[0].set_ylim((0,30))
# add legend to the last subplot
axes[-1].legend(
    (post1, prior, post2, known),
    ( 'апостериорное с равномерным априорным',
      'информативное априорное',
      'апостериорное c информативным априорным',
     r'известная доля женщин в популяции ($\theta=0.485$)'),
    loc='upper center',
    bbox_to_anchor=(0.5, -0.2)
);

# Методы семплирования
Мы рассмотрим два метода семлирования (получения повторной выборки), которые вы можете использовать для оценки качества ваших моделей машинного обучения:
* Кросс-валидация
* Бутстрап

In [ ]:
csv_url = 'https://raw.githubusercontent.com/dsindy/kaggle-titanic/master/data/train.csv'
df = pd.read_csv(csv_url)
df.columns=['psgr','srv','pcls','name','sex','age','sibsp','parch','tkt','fare','cab','emb']
df.sex = 1*(df.sex=='male')

In [ ]:
df_filtered = df.drop(['cab', 'tkt', 'name'], axis=1)

df_filtered["emb_is_S"] = (df_filtered["emb"] == "S").astype(int)
df_filtered["emb_is_C"] = (df_filtered["emb"] == "C").astype(int)

df_filtered = df_filtered.drop(["emb"], axis=1)

median_age = df_filtered["age"].median()
print("Filling NA values with age", median_age)
df_filtered = df_filtered.fillna(median_age)

fare_max = df_filtered['fare'].quantile(0.95)
df_filtered.loc[df_filtered['fare'] > fare_max, "fare"] = fare_max

sibsp_max = df_filtered['sibsp'].quantile(0.95)
df_filtered.loc[df_filtered['sibsp'] > sibsp_max, "sibsp"] = sibsp_max

In [ ]:
df_filtered.head()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    df_filtered.drop(["srv"], axis=1), df_filtered["srv"])

model = KNeighborsClassifier(n_neighbors=3)
model.fit(X_train, y_train)
y_pred = model.predict_proba(X_test)[:, 1]
score = roc_auc_score(y_test, y_pred)
print('ROC AUC', score)

Мы получили показатель ROC AUC для нашего классификатора. Однако если мы повторим запуск этой ячейки несколько раз, то получим другие результаты.
Давайте подтвердим это:

In [ ]:
sns.set()
plt.rcParams['figure.figsize'] = [16, 6]

scores = []
for i in range(250):
    X_train, X_test, y_train, y_test = train_test_split(
        df_filtered.drop(["srv"], axis=1), df_filtered["srv"], test_size=0.3 )

    model = KNeighborsClassifier(n_neighbors=3)
    model.fit(X_train, y_train)
    y_pred = model.predict_proba(X_test)[:, 1]
    scores.append(roc_auc_score(y_test, y_pred))

plt.figure()
sns.boxplot(x=scores)
plt.title('Распределение метрики ROC AUC')
plt.xlabel('ROC AUC')
plt.show()

In [ ]:
plt.figure()
sns.histplot(scores, kde=True)
plt.title('Распределение метрики ROC AUC')
plt.xlabel('ROC AUC')
plt.show()

Оценки метрики могут отличаться более, чем на 0.1 по абсолютной величине между разными прогонами!

Производительность модели в значительной степени зависит от того, как мы выбрали обучающую и тестовую выборки.

Поэтому мы не можем использовать разделение на обучающую и тестовую выборки для получения надежных оценок производительности модели. Это также означает, что мы не можем использовать их для поиска лучших гиперпараметров (например, параметра $k$ для KNN).

Мы хотим знать, насколько хороша наша модель **в целом**.  Мы хотели бы получить метод, который не зависит от способа разбиения данных. Такой метод существует, и он называется кросс-валидацией.

## Кросс-валидация

Идея кросс-валидации:
1. Выполнить несколько разбиений.
2. Для каждого разбиения подогнать модель к обучающей выборке, предсказать и вычислить оценку (например, точность) на тестовой выборке.
3. Усреднить результаты, получить доверительные интервалы.

Существует множество способов выполнения первого шага. Главное, чтобы каждое наблюдение появилось хотя бы в одной из подвыборок.

Иногда можно сделать столько разбиений, сколько у вас наблюдений ($N$). Но это очень медленно, потому что вам придется подгонять $N$ моделей. Поэтому вместо этого вы можете сделать несколько непересекающихся разбиений с $k < N$ наблюдений в каждом, чтобы подогнать меньше моделей и получить похожие результаты. Мы попробуем эти и другие методы ниже.

Кросс-валидация может использоваться для:
* Получения надежной оценки ошибки в тестовой выборке.
* Сравнения моделей по производительности.
* Нахождения наилучших значений для гиперпараметров.


In [ ]:
# Отсюда: https://scikit-learn.org/stable/auto_examples/model_selection/plot_cv_indices.html
def plot_cv_indices(cv, X, y, group, ax, n_splits, lw=10):
    """Create a sample plot for indices of a cross-validation object."""

    # Generate the training/testing visualizations for each CV split
    for ii, (tr, tt) in enumerate(cv.split(X=X, y=y, groups=group)):
        # Fill in indices with the training/test groups
        indices = np.array([np.nan] * len(X))
        indices[tt] = 1
        indices[tr] = 0

        # Visualize the results
        ax.scatter(range(len(indices)), [ii + .5] * len(indices),
                   c=indices, marker='_', lw=lw, cmap=cmap_cv,
                   vmin=-.2, vmax=1.2)

    # Plot the data classes and groups at the end
    ax.scatter(range(len(X)), [ii + 1.5] * len(X),
               c=y, marker='_', lw=lw, cmap=cmap_data)

    ax.scatter(range(len(X)), [ii + 2.5] * len(X),
               c=group, marker='_', lw=lw, cmap=cmap_data)

    # Formatting
    yticklabels = list(range(n_splits)) + ['class', 'group']
    ax.set(yticks=np.arange(n_splits+2) + .5, yticklabels=yticklabels,
           xlabel='Sample index', ylabel="CV iteration",
           ylim=[n_splits+2.2, -.2], xlim=[0, 100])
    ax.set_title('{}'.format(type(cv).__name__), fontsize=15)
    return ax


np.random.seed(0)
cmap_data = plt.cm.Paired
cmap_cv = plt.cm.coolwarm
n_splits = 4

In [ ]:
def draw_bars(bars, y=0.5, vmin=-.2, vmax=1.2, label=None):
    ax = plt.gca()
    ax.scatter(range(len(bars)),  [y] * len(bars), c=bars, marker='_',
               lw=50, label=label, cmap=cmap_cv, vmin=vmin, vmax=vmax)

## Визуализация разделения на train и test

Прежде всего, давайте посмотрим на наше разделение на тренировочную и тестовую выборки.

In [ ]:
train_idx, test_idx = train_test_split(df_filtered.index, test_size=0.3, shuffle=False)
idx_labels = np.array([(idx in train_idx) for idx in df_filtered.index])

train_idx, test_idx = train_test_split(df_filtered.index, test_size=0.3)
idx_labels_shuffled = np.array([(idx in train_idx) for idx in df_filtered.index])

plt.figure()
draw_bars(idx_labels, y=0.5)
draw_bars(idx_labels_shuffled, y=1.5)
plt.xlabel('Observation index')
plt.yticks([0.5, 1.5], ['Train-test split without shuffling', 'Train-test split shuffled'])
plt.legend([Patch(color=cmap_cv(.8)), Patch(color=cmap_cv(.02))],
              ['Training set', 'Testing set'], loc=(1.02, .8))

plt.ylim([0, 2])
plt.title('How train-test split distributes observations')
plt.show()

## K-Fold Кросс-валидация

Идея: разделить набор данных на $K$ **непересекающихся** групп. Каждая из $K$ групп используется в качестве валидационного набора один раз, а остальные $K-1$ групп используются в качестве обучающего набора.

По умолчанию перетасовка данных не производится, но обычно ее полезно сделать.

In [ ]:
def plot_cv_splits(cv, index, title, y=None, groups=None):
    plt.figure()

    y_ticks_pos = []
    y_ticks = []
    for i, (train_idx, test_idx) in enumerate(cv.split(index, y=y, groups=groups)):
        idx_labels = np.array([(idx in train_idx) for idx in index])
        draw_bars(idx_labels, y=0.5+i)
        y_ticks_pos.append(0.5+i)
        y_ticks.append(f'Fold {i}')

    if y is not None:
        i += 1
        draw_bars(y, y=0.5+i, vmin=0.5, vmax=0.8)
        y_ticks_pos.append(0.5+i)
        y_ticks.append(f'Class')

    plt.xlabel('Observation index')
    plt.yticks(y_ticks_pos, y_ticks)
    plt.legend([Patch(color=cmap_cv(.8)), Patch(color=cmap_cv(.02))],
                ['Training set', 'Testing set'], loc=(1.02, .8))

    plt.ylim([0, i+1])
    plt.title(title)
    plt.show()

In [ ]:
cv = KFold(n_splits, shuffle=False)
plot_cv_splits(cv, df_filtered.index, 'How K-Fold distributes observations')

K-Fold великолепен, но давайте посмотрим, как он справляется с классами "Титаника". Я сортирую цели так, чтобы график был более наглядным.

In [ ]:
plot_cv_splits(cv, df_filtered.index, 'How K-Fold handles an imbalanced dataset', y=sorted(df_filtered['srv']))

Это не очень хорошо: в фолдах будет разное распределение классов. Мы хотим сохранить распределение классов, чтобы наши модели могли научиться чему-то действительно полезному.

**Стратифицированный K-fold** гарантирует, что исходное распределение классов сохраняется в каждом фолде.

In [ ]:
cv = StratifiedKFold(n_splits)
plot_cv_splits(cv, df_filtered.index, 'How Startified K-Fold handles an imbalanced dataset', y=sorted(df_filtered['srv']))

# Bootstrap

Бутстреп - это общая статистическая техника, использующая **выборку с заменой** для получения оценок статистических данных, таких как среднее и дисперсия, вычисления доверительных интервалов и многого другого. Он работает **независимо от базового распределения**, поэтому подходит для негауссовых вещей. Представьте себе, как это здорово.

Идея бутстрапа: использовать множество (тысячи) малых выборок для оценки величин всей популяции.

Алгоритм получения бутстреп-оценки для среднего значения:
1. Выберите количество бутстреп-выборок для выполнения.
2. Выберите размер выборки.
3. Для каждой бутстрап-выборки постройте выборку с заменой с выбранным размером.
4. Вычислите среднее значение выборки и сохраните его.
5. Вычислите среднее значение вычисленных выборочных средних.

In [ ]:
column = 'age'
column_sample = df_filtered[column][::10].values
print('Length of source', len(df_filtered[column]))
print('Length of sample', len(column_sample))
sns.distplot(df_filtered[column], label=column)
sns.distplot(column_sample, label='Sample')
plt.legend()
plt.title('Distributions of fare and sample of fare')
plt.show()
print('Source values', len(df_filtered[column]))
print('Sample values', len(column_sample))

print('Source mean', df_filtered[column].mean())
print('Sample mean', column_sample.mean())

Мы заинтересованы в получении оценки среднего возраста и доверительного интервала для него.

In [ ]:
sample_size = len(column_sample)//2
sample = np.random.choice(column_sample, size=sample_size, replace=True)

In [ ]:
n_iterations = 1000
sample_size = len(column_sample)//2
means = []
for i in range(n_iterations):
    sample = np.random.choice(column_sample, size=sample_size, replace=True)
    means.append(np.mean(sample))
means = np.array(means)
print('Bootstrap mean', np.mean(means))

Для получения доверительных интервалов мы можем использовать **метод бутстреп-процентилей** выборочных средних.

Пусть $\theta$ - это бутстреп-средние выборочной совокупности.

Для некоторой ширины доверительного интервала $p$, например, $95\%$:
1. Вычислите $\alpha$ как $1 - p$.
2. Получите нижний перцентиль $\alpha / 2$.
3. Получите верхний перцентиль $1 - \alpha/2$
4. Получите доверительный интервал: $[ процентиль(\theta, \alpha / 2), процентиль(\theta, 1- \alpha / 2) ]$

In [ ]:
p = 0.95
alpha = 1 - p
lower = np.percentile(means, (alpha/2)*100)
upper = np.percentile(means, (1-alpha/2)*100)
print(f'{round(p*100)}% confidence interval for the bootstrap mean: ({lower:2f}, {upper:2f})')

Этот метод является самым простым и, скорее всего, [самым худшим](https://stats.stackexchange.com/questions/355781/is-it-true-that-the-percentile-bootstrap-should-never-be-used). Известно, что он дает плохие результаты, когда распределение выборочной статистики перекошено.

Менее предвзятым методом является **эмпирический бутстрап**. В нем для оценки доверительного интервала используется отклонение выборочных средних от среднего выборочного значения.

1. Вычислите отклонения выборочных средних от среднего значения $\delta = \{\theta_0 - \mathop{\mathbb{E}}[\theta], \theta_1 - \mathop{\mathbb{E}}[\theta], \dots \}$.
2. Получите доверительный интервал: $[ \mathop{\mathbb{E}}[\theta] - процентиль(\delta, 1 - \alpha / 2), \mathop{\mathbb{E}}[\theta] - процентиль(\delta, \alpha / 2) ]$

In [ ]:
deviations = means - np.mean(means)
lower = np.mean(means) - np.percentile(deviations, (1-alpha/2)*100)
upper = np.mean(means) - np.percentile(deviations, (alpha/2)*100)
print(f'Empirical bootstrap {round(p*100)}% confidence interval for the bootstrap mean: ({lower:2f}, {upper:2f})')

## Получение доверительного интервала для точности классификатора с помощью бутстрепа

Представьте, что существует некоторая неизвестная совокупность погрешностей нашего классификатора. Мы хотим получить среднее значение этой совокупности. У нас есть "образцы" наших точностей: результаты подгонки классификатора к некоторым данным и вычисления точности на проверочном множестве.

In [ ]:
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(
    df_filtered.drop(["srv"], axis=1), df_filtered["srv"], test_size=0.3)

X_train = X_train.values
y_train = y_train.values

pipe = Pipeline([('scale', StandardScaler()),
                ('clf', KNeighborsClassifier(n_jobs=-1,))])

In [ ]:
from tqdm import tqdm
from sklearn.metrics import accuracy_score
from sklearn.utils import resample

n_iterations = 100
n_size = len(X_train)//2
model = pipe

accuracies = []
for i in tqdm(range(n_iterations)):
	train_idx = resample(range(len(X_train)), n_samples=n_size)
	val_idx = np.array([i for i in range(len(X_train)) if i not in train_idx])

	model.fit(X_train[train_idx], y_train[train_idx])
	# evaluate model
	predictions = model.predict(X_train[val_idx])
	score = accuracy_score(y_train[val_idx], predictions)
	accuracies.append(score)

sns.displot(accuracies)
plt.title('Accuracies on bootstrap samples')
plt.show()

In [ ]:
p = 0.95
alpha = 1 - p
lower = np.percentile(accuracies, (alpha/2)*100)
upper = np.percentile(accuracies, (1-alpha/2)*100)
print(f'Accuracy according to percentile bootstrap: ({lower:2f}, {upper:2f})')

In [ ]:
model.fit(X_train, y_train)
predictions = model.predict(X_test)
accuracy = accuracy_score(y_test, predictions)
print(f'True accuracy on test set: {accuracy:2f}')